# Model Training

In [16]:
import sys
import os

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

import torch
import torch.nn as nn
from models.models import TinyEEGCNN, TrainConfig
from data.dataset import CustomEEGDataset
from torch.utils.data import DataLoader, random_split

In [17]:
def train_one_epoch(model, loader, optimizer, loss_fn, device, grad_clip=1.0):
    model.train()
    total_loss = 0.0
    for x, y in loader:
        x = x.to(device)                  # [B, 1, L]
        y = y.to(device).float().unsqueeze(1)  # [B, 1]
        optimizer.zero_grad()
        logit = model(x)                  # [B, 1]
        loss = loss_fn(logit, y)
        loss.backward()
        if grad_clip is not None:
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

@torch.no_grad()
def infer_probs(model, loader, device):
    model.eval()
    probs, ys = [], []
    for x, y in loader:
        x = x.to(device)
        logit = model(x)
        p = torch.sigmoid(logit).squeeze(1).cpu().numpy()
        probs.append(p)
        ys.append(y.numpy())
    return np.concatenate(ys), np.concatenate(probs)

In [21]:
cfg = TrainConfig()

eeg_dataset = CustomEEGDataset(cfg.csv_path)
train_dataset, test_dataset = random_split(eeg_dataset, [0.8, 0.2])
train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False)


print(train_dataset[0])
# pos = (train_dataset.y == 1).sum()
# neg = (train_dataset.y == 0).sum()
# pos_weight = torch.tensor([neg / max(pos, 1)], dtype=torch.float32, device=cfg.device)
# print(pos_weight)
# model = TinyEEGCNN()



NameError: name 'torch' is not defined